In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d
import os

In [3]:
# Utility Functions

def display_image(img):
    img2 = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img2)
    plt.show()

def draw_line(img, A, B):
    cv2.line(img, A, B, (0, 255, 0), thickness=3, lineType=8)

def add_column(original, newcol):
    return np.concatenate([original, newcol], axis=1)

def add_row(original, newrow): # didn't use
    return np.concatenate([original, newrow], axis=0)

def camera_project(P, world_point):
    # Convert the points in 3d space to homogeneous coordinates by appending 1, and making it a column vector
    homo_point = np.transpose(np.array([world_point[0], world_point[1], world_point[2], 1]))

    #Projecting the points into 2d by applying the camera projection matrix
    val = np.matmul(P, homo_point)

    # Returned pixel coordinates, homogenous(a, b, c) => (a/c, b/c) in pixels.
    # int as pixels are integral.
    return (int(val[0] / val[2]), int(val[1] / val[2]))

def compute_homography(lstpts): # Calculates homography matrix b/w initial and final points
    x1 = lstpts[0][0][0]
    x2 = lstpts[1][0][0]
    x3 = lstpts[2][0][0]
    x4 = lstpts[3][0][0]
    y1 = lstpts[0][0][1]
    y2 = lstpts[1][0][1]
    y3 = lstpts[2][0][1]
    y4 = lstpts[3][0][1]
    x1f = lstpts[0][1][0]
    x2f = lstpts[1][1][0]
    x3f = lstpts[2][1][0]
    x4f = lstpts[3][1][0]
    y1f = lstpts[0][1][1]
    y2f = lstpts[1][1][1]
    y3f = lstpts[2][1][1]
    y4f = lstpts[3][1][1]
    ans = np.array([0, 0, 0, 0, 0, 0, 0, 0, 1])
    q = np.array([
        [-1*x1, -y1, -1, 0, 0, 0, x1*x1f, y1*x1f, x1f],
        [0, 0, 0, -1*x1, -1*y1, -1, x1*y1f, y1*y1f, y1f],
        [-1*x2, -y2, -1, 0, 0, 0, x2*x2f, y2*x2f, x2f],
        [0, 0, 0, -1*x2, -1*y2, -1, x2*y2f, y2*y2f, y2f],
        [-1*x3, -y3, -1, 0, 0, 0, x3*x3f, y3*x3f, x3f],
        [0, 0, 0, -1*x3, -1*y3, -1, x3*y3f, y3*y3f, y3f],
        [-1*x4, -y4, -1, 0, 0, 0, x4*x4f, y4*x4f, x4f],
        [0, 0, 0, -1*x4, -1*y4, -1, x4*y4f, y4*y4f, y4f],
        [0, 0, 0, 0, 0, 0, 0, 0, 1]
    ])
    H = list(np.linalg.solve(q,ans))
    H = np.array([H[:3], H[3:6], H[6:9]])
    return H

def draw_point(image, point, color):
    if color == "green":
        cv2.circle(image, point, 10, (0, 255, 0), thickness=3, lineType=8)
    elif color == "red":
        cv2.circle(image, point, 8, (0, 0, 255), thickness=3, lineType=8)
        
def create_output(vertices, colors, filename): # taken from github
    colors = colors.reshape(-1,3)
    vertices = np.hstack([vertices.reshape(-1,3),colors])

    ply_header = '''ply
        format ascii 1.0
        element vertex %(vert_num)d
        property float x
        property float y
        property float z
        property uchar red
        property uchar green
        property uchar blue
        end_header
        '''
    with open(filename, 'w') as f:
        f.write(ply_header %dict(vert_num=len(vertices)))
        np.savetxt(f,vertices,'%f %f %f %d %d %d')

In [5]:
lst = os.listdir('./data/img2')
# imgs = ['0000000460.png', '0000000461.png']
# iml = cv2.imread('./img2/0000000460.png')
# imr = cv2.imread('./img3/0000000460.png')
imgs = sorted(lst)[:1]
print(imgs)

['0000000460.png']


In [6]:
pose = np.array([
    np.array([[-9.098548e-01,5.445376e-02,-4.113381e-01,-1.872835e+02],[4.117828e-02,9.983072e-01,4.107410e-02,1.870218e+00],[4.128785e-01,2.043327e-02,-9.105569e-01,5.417085e+01]]),
    np.array([[-9.283201e-01,4.509830e-02,-3.690366e-01,-1.874227e+02],[3.357747e-02,9.987291e-01,3.758534e-02,1.883289e+00],[3.702626e-01,2.249990e-02,-9.286546e-01,5.369416e+01]]),
    np.array([[-9.452512e-01,3.706129e-02,-3.242324e-01,-1.875256e+02],[2.776133e-02,9.990610e-01,3.326342e-02,1.887417e+00],[3.251607e-01,2.244117e-02,-9.453925e-01,5.321309e+01]]),
    np.array([[-9.602059e-01,3.410654e-02,-2.772028e-01,-1.876307e+02],[2.674480e-02,9.991831e-01,3.029615e-02,1.893359e+00],[2.780097e-01,2.167680e-02,-9.603337e-01,5.273710e+01]]),
    np.array([[-9.729422e-01,3.276894e-02,-2.287129e-01,-1.876915e+02],[2.667847e-02,9.992036e-01,2.967147e-02,1.900316e+00],[2.295031e-01,2.276691e-02,-9.730416e-01,5.225276e+01]]),
    np.array([[-9.831529e-01,3.240420e-02,-1.798899e-01,-1.877485e+02],[2.738709e-02,9.991654e-01,3.030454e-02,1.901338e+00],[1.807218e-01,2.486734e-02,-9.832198e-01,5.177232e+01]]),
    np.array([[-9.911510e-01,2.523874e-02,-1.303180e-01,-1.877723e+02],[2.111370e-02,9.992343e-01,3.293915e-02,1.900778e+00],[1.310496e-01,2.989617e-02,-9.909249e-01,5.129098e+01]]),
    np.array([[-9.966637e-01,1.446740e-02,-8.032513e-02,-1.877414e+02],[1.146235e-02,9.992216e-01,3.774707e-02,1.901606e+00],[8.080871e-02,3.670042e-02,-9.960537e-01,5.080341e+01]]),
    np.array([[-9.995040e-01,1.006074e-02,-2.984320e-02,-1.877165e+02],[8.815319e-03,9.990965e-01,4.157437e-02,1.903912e+00],[3.023451e-02,4.129067e-02,-9.986896e-01,5.033042e+01]]),
    np.array([[-9.996705e-01,1.307490e-02,2.209183e-02,-1.876828e+02],[1.395603e-02,9.990937e-01,4.021250e-02,1.909578e+00],[-2.154603e-02,4.050756e-02,-9.989469e-01,4.986716e+01]]),
    np.array([[-9.970705e-01,2.051987e-02,7.368440e-02,-1.876045e+02],[2.335961e-02,9.990090e-01,3.788635e-02,1.913879e+00],[-7.283396e-02,3.949659e-02,-9.965617e-01,4.940375e+01]]),
    np.array([[-9.919546e-01,2.316199e-02,1.244574e-01,-1.875198e+02],[2.767742e-02,9.990153e-01,3.467489e-02,1.916071e+00],[-1.235318e-01,3.784057e-02,-9.916189e-01,4.895472e+01]]),
    np.array([[-9.843986e-01,2.362611e-02,1.743596e-01,-1.873856e+02],[2.975169e-02,9.990255e-01,3.260177e-02,1.916040e+00],[-1.734194e-01,3.728062e-02,-9.841422e-01,4.850872e+01]]),
    np.array([[-9.745046e-01,2.207656e-02,2.232787e-01,-1.872454e+02],[2.939030e-02,9.991330e-01,2.948581e-02,1.919502e+00],[-2.224342e-01,3.529628e-02,-9.743086e-01,4.808322e+01]]),
    np.array([[-9.620042e-01,2.331542e-02,2.720374e-01,-1.870919e+02],[3.162970e-02,9.991557e-01,2.621754e-02,1.921924e+00],[-2.711965e-01,3.382584e-02,-9.619295e-01,4.766758e+01]]),
    np.array([[-9.466844e-01,2.747540e-02,3.209888e-01,-1.869050e+02],[3.735473e-02,9.989978e-01,2.465901e-02,1.925132e+00],[-3.199896e-01,3.533474e-02,-9.467619e-01,4.724980e+01]]),
    np.array([[-9.303521e-01,3.021656e-02,3.654203e-01,-1.867142e+02],[4.116827e-02,9.989052e-01,2.221412e-02,1.922721e+00],[-3.643491e-01,3.571067e-02,-9.305775e-01,4.684089e+01]]),
    np.array([[-9.132370e-01,2.475193e-02,4.066762e-01,-1.864782e+02],[3.739334e-02,9.990320e-01,2.316585e-02,1.917552e+00],[-4.057092e-01,3.636289e-02,-9.132786e-01,4.643056e+01]]),
    np.array([[-8.953418e-01,1.994046e-02,4.449332e-01,-1.862433e+02],[3.564541e-02,9.990008e-01,2.695748e-02,1.910636e+00],[-4.439511e-01,3.999598e-02,-8.951580e-01,4.602498e+01]]),
    np.array([[-8.776813e-01,1.983338e-02,4.788341e-01,-1.859999e+02],[3.901401e-02,9.987839e-01,3.014110e-02,1.906858e+00],[-4.776541e-01,4.513551e-02,-8.773878e-01,4.562057e+01]]),
    np.array([[-8.608718e-01,2.164481e-02,5.083614e-01,-1.857437e+02],[4.407236e-02,9.985119e-01,3.211896e-02,1.904895e+00],[-5.069097e-01,5.005498e-02,-8.605447e-01,4.521752e+01]])
])
len(pose)

21

In [12]:
def make_new_pt(pt_3d, C):
    P = np.array([
        [C[0][0], C[0][1], C[0][2], C[0][3]],
        [C[1][0], C[1][1], C[1][2], C[1][3]],
        [C[2][0], C[2][1], C[2][2], C[2][3]],
    ])
    pt = np.mat([pt_3d[0], pt_3d[1], pt_3d[2], 1]).T
    mat = np.matmul(P, pt)
    return mat

combination = []
A0 = np.vstack([pose[0], np.array([0, 0, 0, 1])])
combination.append(A0)
for c in range(1, len(pose)):
    A = np.vstack([pose[c], np.array([0, 0, 0, 1])])
    combination.append(A)

final_output = []
final_color = []
output_colors = []
output_points = []
Height = 0
Width = 0
def main():
    global final_output, final_color, output_colors, output_points
    focal_length = 7.070912e+02
    k = np.float32([[7.070912e+02, 0.000000e+00, 6.018873e+02], 
         [0.000000e+00, 7.070912e+02, 1.831104e+02], 
         [0.000000e+00, 0.000000e+00, 1.000000e+00]])
    baseline = 0.53790448812

    for count, img in enumerate(imgs):
        iml = cv2.imread('./data/img2/' + img)
        imr = cv2.imread('./data/img3/' + img)
        window_size = 3
        min_disp = 16
        num_disp = 112-min_disp
        stereo = cv2.StereoSGBM_create(minDisparity = min_disp,
           numDisparities = num_disp+(2) * 16,
           blockSize = 5,
           P1 = 8*3*window_size**2,
           P2 = 32*3*window_size**2,
           disp12MaxDiff = 1,
           uniquenessRatio = 12,
           speckleWindowSize = 400,
           speckleRange = 5
       )
        disparity_map = stereo.compute(iml, imr).astype(np.float32) / 16.0
        h, w = iml.shape[:2]
        Height = h
        Width = w
        Q = np.float32([[1, 0, 0, -0.5*w],
                        [0,-1, 0,  0.5*h], 
                        [0, 0, 0, -focal_length], 
                        [0, 0, 1/baseline,  0]])
        points = cv2.reprojectImageTo3D(disparity_map, Q)
        colors = cv2.cvtColor(iml, cv2.COLOR_BGR2RGB)
        
# Hand coding reprojectImageto3D....didn't give us very good results
# Overwriting colors and points in this process with own implementation
# 
# 
#         for i in range(len(iml)):
#             for j in range(len(iml[i])):
#                 colors[i][j] = iml[i][j]
#                 pt_temp = Q @ np.array([i, j, disparity_map[i][j], 1]).T
#                 W = pt_temp[3]
#                 to_ap = [pt_temp[0]/W, pt_temp[1]/W, pt_temp[2]/W]
#                 points[i][j] = to_ap

        print(points.shape)
        print(colors.shape)
        mask = disparity_map > disparity_map.min()
        output_points = points[mask]
        output_colors = colors[mask]
        for i,pt in enumerate(output_points):
            final_output.append(np.array(make_new_pt(pt, combination[count]).T).astype('float32')[0])
            final_color.append(output_colors[i])
        print("Length of output Points:", len(output_points), "count:", count)
    final_output = np.array(final_output)
    final_color = np.array(final_color)
    print(final_output.shape, final_color.shape)
    create_output(final_output, final_color, 'jnrecon_mult.ply')
main()

(370, 1226, 3)
(370, 1226, 3)
Length of output Points: 304696 count: 0
(304696, 3) (304696, 3)


In [15]:
# p3p part
#d = (f - Z' * b ) / ( Z' * a)

#Ix = X' * ( d * a + b ) + Cx

#Iy = Y' * ( d * a + b ) + Cy
Height = 370
Width = 1226
def find_impts(X, Y, Z):
    f = 7.070912e+02
    baseline = 0.53790448812
    a = -1/baseline
    b = 0
    d = (f - Z * b) / (Z * a)
    Ix = X * (d*a + b) + 0.5*Width
    Iy = Y * (d*a + b) + 0.5*Height
    return (int(Iy), int(Ix))

In [17]:
inv_c = []
hold_pts = []
for c in combination:
    inv_c.append(np.linalg.inv(c))
for i, pt in enumerate(final_output):
    pose_num = int(i/304696)
    temp_pt = np.array([pt[0], pt[1], pt[2], 1]).T
    out_pt = np.matmul(inv_c[pose_num], temp_pt)
    hold_pts.append(find_impts(out_pt[0], out_pt[1], out_pt[2]))